In [7]:
# 从原始 OSM 区域中获取系统的元素样本
# -*- coding: utf-8 -*-
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "C:/Users/lk235/Desktop/Project_1/wenzhou.osm"  # Replace this with your osm file
SAMPLE_FILE = "C:/Users/lk235/Desktop/Project_1/sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')
    print 'WRITE END'

WRITE END


In [8]:
# 在地图中的每个类别有多少数据
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict
import re
import os

SAMPLE_FILE = "C:/Users/lk235/Desktop/Project_1/sample.osm"


       
def count_tags(filename):
    osm_file = open(filename, "r")
    tags = {}
    pattern = re.compile("'(.*)'")
    for event, elem in ET.iterparse(osm_file):
        k = str(pattern.findall(str(elem))).strip('[]').strip('\'')
        # k = pattern.findall(str(elem))
        
        if k not in tags.keys():
            tags[k] = 1
        else:
            tags[k] = tags[k] + 1
        
    return tags
print 'start'
print count_tags(SAMPLE_FILE)
print os.linesep


start


{'node': 42224, 'nd': 49410, 'member': 350, 'tag': 8129, 'relation': 18, 'way': 2749, 'osm': 1}




In [28]:
# 审查路名：way节点下发现名字中有非中文、乱码等情况、非温州道路;如果英文路名存在中文字段，则用中文字段替换
# node节点下名字非中文，uu等无法理解的名字；china-class字段全部为英文或拼音，应统一改成中文
# 审查代码如下：
import sys
reload(sys)
sys.setdefaultencoding('utf8')

SAMPLE_FILE = "C:/Users/lk235/Desktop/Project_1/sample.osm"
# def is_street_name(elem):
#     return (elem.attrib['k'] == "addr:street")
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    count = 0
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        # if elem.tag == "node" or elem.tag == "way":
        if elem.tag == "node":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == 'place':
                    # if  tag.attrib['v'].find('台州') == -1:
                        count = count + 1
                        print tag.attrib['v']
                # if is_street_name(tag):
              
                    # audit_street_type(street_types, tag.attrib['v'])
    print count               
    osm_file.close()
    return street_types

audit(SAMPLE_FILE)